**웹캠** 

In [8]:
import cv2
import numpy as np
from keras.models import load_model
from keras.layers import DepthwiseConv2D
import requests
import json
from bs4 import BeautifulSoup

def food_info(names):
    query = '+'.join(names)
    url = f"https://www.10000recipe.com/recipe/list.html?q={query}"
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else:
        print("HTTP 응답 오류:", response.status_code)
        return

    food_list = soup.find_all(attrs={'class': 'common_sp_link'})
    if not food_list:
        print("식재료에 맞는 레시피가 없습니다.")
        return

    recipes = []

    for food in food_list[:5]:
        food_id = food['href'].split('/')[-1]
        new_url = f'https://www.10000recipe.com/recipe/{food_id}'
        new_response = requests.get(new_url)
        if new_response.status_code == 200:
            html = new_response.text
            soup = BeautifulSoup(html, 'html.parser')
        else:
            print("HTTP 응답 오류:", new_response.status_code)
            continue

        food_info = soup.find(attrs={'type': 'application/ld+json'})
        if food_info is not None:
            result = json.loads(food_info.text)
            ingredients = result.get('recipeIngredient', [])
            recipe = [step['text'] for step in result.get('recipeInstructions', [])]
            for i in range(len(recipe)):
                recipe[i] = f'{i + 1}. ' + recipe[i]

            view_count_element = soup.find(attrs={'class': 'view_count'})
            view_count = int(view_count_element.text.replace(',', '')) if view_count_element else 0

            print(f"레시피: {soup.title.string}, 조회수: {view_count}")

            recipes.append({
                'title': soup.title.string,
                'ingredients': ingredients,
                'recipe': recipe,
                'view_count': view_count
            })

    if recipes:
        best_recipe = max(recipes, key=lambda x: x['view_count'])

        res = {
            'name': best_recipe['title'],
            'ingredients': ', '.join(best_recipe['ingredients']),
            'recipe': best_recipe['recipe']
        }
        return res
    else:
        print("적합한 레시피를 찾지 못했습니다.")
        return

# 모델과 라벨 로드
model = load_model("C:/Users/youngjin/Desktop/cabdi2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

class_names = open("C:/Users/youngjin/Desktop/cabdi2024/labels.txt", "r").readlines()

# 웹캠 초기화
camera = cv2.VideoCapture(0)
is_recognizing = True
predicted_classes = []

while True:
    ret, image = camera.read()
    if not ret:
        print("웹캠 이미지 캡처 실패")
        break

    resized_image = cv2.resize(image, (224, 224))
    cv2.imshow("Webcam Image", resized_image)

    if is_recognizing:
        image = np.asarray(resized_image, dtype=np.float32)
        image = np.expand_dims(image, axis=0)
        image = image / 255.0

        prediction = model.predict(image)
        index = np.argmax(prediction)
        class_name = class_names[index].strip()
        confidence_score = prediction[0][index]

        print("Class:", class_name[2:], "Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")

        if confidence_score > 0.5:  # 낮은 신뢰도 예측을 피하기 위한 임계값 설정
            print(f"인식된 음식: {class_name[2:]}. 맞습니까? (y/n)")
            while True:
                keyboard_input = cv2.waitKey(1)
                if keyboard_input == ord('y'):
                    predicted_classes.append(class_name[2:])
                    print("추가할 식재료가 있습니까? (y/n)")
                    while True:
                        additional_input = cv2.waitKey(1)
                        if additional_input == ord('y'):
                            is_recognizing = True
                            break
                        elif additional_input == ord('n'):
                            is_recognizing = False
                            break
                        elif additional_input == 27:  # ESC 키를 누르면 종료
                            is_recognizing = False
                            break
                    break
                elif keyboard_input == ord('n'):
                    is_recognizing = True  # 인식을 계속 진행
                    break
                elif keyboard_input == 27:  # ESC 키를 누르면 종료
                    is_recognizing = False
                    break

    if not is_recognizing:
        break

camera.release()
cv2.destroyAllWindows()

if predicted_classes:
    print("예측된 식재료:", " ".join(predicted_classes))
    food_info_result = food_info(predicted_classes)
    if food_info_result:
        print("추천 레시피:")
        print("Food Name:", food_info_result["name"])
        print("Ingredients:", ", ".join(predicted_classes))
        print("Recipe:")
        for step in food_info_result['recipe']:
            print(step)
else:
    print("식재료를 인식하지 못했습니다.")


56
55


TypeError: Error when deserializing class 'DepthwiseConv2D' using config={'name': 'expanded_conv_depthwise', 'trainable': True, 'dtype': 'float32', 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'linear', 'use_bias': False, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'bias_regularizer': None, 'activity_regularizer': None, 'bias_constraint': None, 'depth_multiplier': 1, 'depthwise_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'depthwise_regularizer': None, 'depthwise_constraint': None}.

Exception encountered: Unrecognized keyword arguments passed to DepthwiseConv2D: {'groups': 1}

**아두이노**

In [ ]:
#1
import shutil
from keras.models import load_model
from keras.layers import DepthwiseConv2D
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import serial
import time



# Load the model
model = load_model("C:/Users/youngjin/Desktop/캡디2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

# Load the labels
class_names = open("C:/Users/youngjin/Desktop/캡디2024/labels.txt", "r").readlines()

# Set up serial communication with Arduino
ser = serial.Serial('COM3', 9600)  # Replace 'COM3' with your Arduino's COM port
time.sleep(2)  # Wait for the serial connection to initialize

while True:
    # Read from Arduino
    if ser.in_waiting > 0:
        line = ser.readline().decode('utf-8').rstrip()
        print("Received:", line)
        
        # Assuming the Arduino sends the image data in a specific format
        # For example, if Arduino sends base64 encoded image data, you need to decode it
        try:
            # Decode the base64 string to bytes
            image_data = base64.b64decode(line)
            
            # Convert bytes to a PIL image
            image = Image.open(io.BytesIO(image_data))
            
            # Resize the image to (224, 224) pixels
            resized_image = image.resize((224, 224))
            
            # Convert the image to a numpy array
            image_array = np.asarray(resized_image, dtype=np.float32)
            image_array = np.expand_dims(image_array, axis=0)
            
            # Normalize the image array
            image_array = image_array / 255.0
            
            # Predict the model
            prediction = model.predict(image_array)
            index = np.argmax(prediction)
            class_name = class_names[index]
            confidence_score = prediction[0][index]

            # Print prediction and confidence score
            print("Class:", class_name[2:], end="")
            print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")
        
        except Exception as e:
            print("Error processing image:", e)

ser.close()


In [ ]:
#2
import shutil
from keras.models import load_model
from keras.layers import DepthwiseConv2D
import numpy as np
import tensorflow as tf
from PIL import Image
import io
import serial

def unzip_files(zip_path, extract_path):
    shutil.unpack_archive(zip_path, extract_path)

zip_path1 = 'C:/Users/youngjin/Desktop/캡디2024/Cheese.v1i.createml.zip'
zip_path2 = 'C:/Users/youngjin/Desktop/캡디2024/onion.v1i.createml.zip'
extract_path = 'C:/Users/youngjin/Desktop/캡디2024'

unzip_files(zip_path1, extract_path)
unzip_files(zip_path2, extract_path)

cheese = zip_path1
bread = zip_path2

# Load the model
model = load_model("C:/Users/youngjin/Desktop/캡디2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

# Load the labels
class_names = open("C:/Users/youngjin/Desktop/캡디2024/labels.txt", "r").readlines()

# Setup serial communication
ser = serial.Serial('COM3', 115200, timeout=10)  # Replace 'COM3' with your serial port

def predict_image(image):
    # Preprocess the image for the model
    image = np.asarray(image, dtype=np.float32)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0

    # Predict the image
    prediction = model.predict(image)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    return class_name.strip(), confidence_score

def main():
    while True:
        # Read the image size
        image_size_bytes = ser.read(4)
        if len(image_size_bytes) != 4:
            continue
        
        image_size = int.from_bytes(image_size_bytes, byteorder='little')

        # Read the image data
        image_data = ser.read(image_size)
        if len(image_data) != image_size:
            continue

        # Load the image
        image = Image.open(io.BytesIO(image_data))

        # Predict the image
        class_name, confidence_score = predict_image(image)

        # Print the prediction result
        print(f"Class: {class_name}, Confidence Score: {confidence_score * 100:.2f}%")

if __name__ == "__main__":
    main()


In [ ]:
#3
import shutil
from keras.models import load_model
from keras.layers import DepthwiseConv2D
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import serial
import time

def unzip_files(zip_path, extract_path):
    shutil.unpack_archive(zip_path, extract_path)

zip_path1 = 'C:/Users/youngjin/Desktop/캡디2024/Cheese.v1i.createml.zip'
zip_path2 = 'C:/Users/youngjin/Desktop/캡디2024/onion.v1i.createml.zip'
extract_path = 'C:/Users/youngjin/Desktop/캡디2024'

unzip_files(zip_path1, extract_path)
unzip_files(zip_path2, extract_path)

cheese = zip_path1
bread = zip_path2


# Load the model
model = load_model("C:/Users/youngjin/Desktop/캡디2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

# Load the labels
class_names = open("C:/Users/youngjin/Desktop/캡디2024/labels.txt", "r").readlines()

# Set up serial communication with Arduino
ser = serial.Serial('COM3', 9600)  # Replace 'COM3' with your Arduino's COM port
time.sleep(2)  # Wait for the serial connection to initialize

while True:
    # Read from Arduino
    if ser.in_waiting > 0:
        line = ser.readline().decode('utf-8').rstrip()
        print("Received:", line)
        
        # Assuming the Arduino sends the image data in a specific format
        # For example, if Arduino sends base64 encoded image data, you need to decode it
        try:
            # Convert the image data string to a PIL image
            image = Image.open(io.BytesIO(line))
            
            # Resize the image to (224, 224) pixels
            resized_image = image.resize((224, 224))
            
            # Convert the image to a numpy array
            image_array = np.asarray(resized_image, dtype=np.float32)
            image_array = np.expand_dims(image_array, axis=0)
            
            # Normalize the image array
            image_array = image_array / 255.0
            
            # Predict the model
            prediction = model.predict(image_array)
            index = np.argmax(prediction)
            class_name = class_names[index]
            confidence_score = prediction[0][index]

            # Print prediction and confidence score
            print("Class:", class_name[2:], end="")
            print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")
        
        except Exception as e:
            print("Error processing image:", e)

ser.close()


In [ ]:
#4
import serial
import numpy as np
import io
from PIL import Image
from keras.models import load_model
import tensorflow as tf

# 시리얼 포트 초기화
ser = serial.Serial('COM3', 115200)  # 시리얼 포트와 속도를 맞춰줘야 함
time.sleep(2)  # 시리얼 연결이 초기화될 때까지 대기

# 모델 로드
model = load_model("C:/Users/youngjin/Desktop/캡디2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

# 레이블 로드
class_names = open("C:/Users/youngjin/Desktop/캡디2024/labels.txt", "r").readlines()

def process_image(image_data):
    # 바이트 데이터를 이미지로 변환
    image = Image.open(io.BytesIO(image_data))
    image = image.resize((224, 224))
    image_array = np.asarray(image, dtype=np.float32)
    image_array = np.expand_dims(image_array, axis=0)
    image_array = image_array / 255.0
    return image_array

while True:
    # 시리얼 데이터 수신
    if ser.in_waiting > 0:
        photo_data = ser.read(ser.in_waiting)
        
        # 이미지 데이터 처리
        try:
            processed_image = process_image(photo_data)
            
            # 모델 예측
            prediction = model.predict(processed_image)
            index = np.argmax(prediction)
            class_name = class_names[index]
            confidence_score = prediction[0][index]

            # 예측 결과 출력
            print(f"Class: {class_name[2:]}, Confidence Score: {str(np.round(confidence_score * 100))[:-2]}%")
        
        except Exception as e:
            print(f"Error processing image: {e}")

ser.close()


**url**

In [ ]:
import serial
import numpy as np
import io
from PIL import Image
from keras.models import load_model
import tensorflow as tf
import time

# 시리얼 포트 초기화
ser = serial.Serial('COM3', 115200)  # 시리얼 포트와 속도를 맞춰줘야 함
time.sleep(2)  # 시리얼 연결이 초기화될 때까지 대기

# 모델 로드
model = load_model("C:/Users/youngjin/Desktop/캡디2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

# 레이블 로드
class_names = open("C:/Users/youngjin/Desktop/캡디2024/labels.txt", "r").readlines()

def process_image(image_data):
    # 바이트 데이터를 이미지로 변환
    image = Image.open(io.BytesIO(image_data))
    image = image.resize((224, 224))
    image_array = np.asarray(image, dtype=np.float32)
    image_array = np.expand_dims(image_array, axis=0)
    image_array = image_array / 255.0
    return image_array

def predict_image():
    while True:
        # 시리얼 데이터 수신
        if ser.in_waiting > 0:
            photo_data = ser.read(ser.in_waiting)
            
            # 이미지 데이터 처리
            try:
                processed_image = process_image(photo_data)
                
                # 모델 예측
                prediction = model.predict(processed_image)
                index = np.argmax(prediction)
                class_name = class_names[index]
                confidence_score = prediction[0][index]

                # 예측 결과 출력 및 반환
                print(f"Class: {class_name[2:]}, Confidence Score: {str(np.round(confidence_score * 100))[:-2]}%")
                return class_name[2:]  # 예측된 클래스 이름 반환
            
            except Exception as e:
                print(f"Error processing image: {e}")

if __name__ == "__main__":
    predicted_class = predict_image()
    ser.close()


In [ ]:
import requests
import json
from bs4 import BeautifulSoup
from predict_image import predict_image  # 모델 예측 함수를 import

def food_info(name):
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else:
        print("HTTP response error:", response.status_code)
        return

    food_list = soup.find_all(attrs={'class':'common_sp_link'})
    if not food_list:
        print("식재료에 맞는 레시피가 없습니다.")
        return

    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)
    if new_response.status_code == 200:
        html = new_response.text
        soup = BeautifulSoup(html, 'html.parser')
    else:
        print("HTTP response error:", new_response.status_code)
        return

    food_info = soup.find(attrs={'type':'application/ld+json'})
    result = json.loads(food_info.text)
    ingredient = ','.join(result['recipeIngredient'])
    recipe = [result['recipeInstructions'][i]['text'] for i in range(len(result['recipeInstructions']))]
    for i in range(len(recipe)):
        recipe[i] = f'{i+1}. ' + recipe[i]

    title = soup.title.string

    res = {
        'name': title,
        'ingredients': ingredient,
        'recipe': recipe
    }

    return res

if __name__ == "__main__":
    predicted_ingredients = predict_image()  # 모델 예측 함수 호출
    if predicted_ingredients:
        ingredients_str = " ".join(predicted_ingredients)  # 예측된 식재료를 공백으로 구분된 문자열로 변환
        food_info_result = food_info(ingredients_str)
        if food_info_result:
            print("Food Name:", food_info_result['name'])
            print("Ingredients:", food_info_result['ingredients'])
            print("Recipe:")
            for step in food_info_result['recipe']:
                print(step)


정확도

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

# 모델 로드
model = load_model('C:/Users/youngjin/Desktop/design/keras_model.h5')

# 이미지 경로
img_path = 'test/예시이미지.jpg'

# 이미지 불러오기 및 전처리
img = image.load_img(img_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255  # 이미지 정규화

# 예측
pred = model.predict(img)
predicted_class = np.argmax(pred)

# 예측 결과 출력
print("Predicted class:", predicted_class)


막대 그래프

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

# 모델 로드
model = load_model('C:/Users/youngjin/Desktop/design/keras_model.h5')

# 이미지 경로
img_path = 'test/예시이미지.jpg'

# 이미지 불러오기 및 전처리
img = image.load_img(img_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255  # 이미지 정규화

# 예측
pred = model.predict(img)
predicted_class = np.argmax(pred)

# 예측 결과 출력
print("Predicted class:", predicted_class)
